In [1]:
import numpy as np
import pandas as pd
from importlib import reload

from bs4 import BeautifulSoup
import urllib
import requests

from tqdm import tqdm_notebook
from tqdm import tqdm
import datetime as dt

import pdaactconn as pc
from trialexplorer import AACTStudySet
from trialexplorer import bing
import pickle
from collections import Counter

import matplotlib.pyplot as plt
%matplotlib inline

tqdm.pandas()

### Example

In [ ]:
feat_doc, feat_links, feat_wiki = bing.bing('non small cell lung cancer')

In [ ]:
feat_doc2, feat_links2, feat_wiki2 = bing.bing("carcinoma, non-small-cell lung")

In [ ]:
print("---STRING 1 ---\n")
print(feat_doc)

print("\n---STRING 2 ---\n")
print(feat_doc2)

In [ ]:
print("---LINK 1 ---\n")
print(feat_links)

print("\n---LINK 2 ---\n")
print(feat_links2)

In [ ]:
feat_wiki, feat_wiki2

# What are all of the terms that we have to classify?

In [2]:
# selecting all interventional studies
conn = pc.AACTConnection(source=pc.AACTConnection.REMOTE)
ss = AACTStudySet.AACTStudySet(conn= conn, tqdm_handler=tqdm_notebook)
ss.add_constraint("study_type = 'Interventional'")
ss.load_studies()

255092 studies loaded!


In [3]:
# loading all dimensional data
ss.add_dimensions('browse_conditions')
ss.add_dimensions('conditions')
ss.refresh_dim_data()

Successfuly added these 1 dimensions: ['browse_conditions']
Failed to add these 0 dimensions: []
Successfuly added these 1 dimensions: ['conditions']
Failed to add these 0 dimensions: []


Syncing the temp table temp_cur_studies in 511 chunks x 500 records each

Creating index on the temp table
 - Loading dimension browse_conditions
 -- Loading raw data
 -- Sorting index
 - Loading dimension conditions
 -- Loading raw data
 -- Sorting index


In [4]:
bc = ss.dimensions['browse_conditions']
c = ss.dimensions['conditions']

In [5]:
len(c.data['name'].unique())

65144

## Features:

How to determine if 2 studies are studing the same condition?
- condition names (lev distance)
- condition bing results (bag of words)
- mesh terms (jaccard distance)
- mesh tree location (tree distance)
- adjective descriptors (such as "chronic")
- type, grade, stage, AJCC (type1, type2) etc.

    

In [6]:
c.data.to_pickle('raw_data/all_conditions.p')

### doing top 100:

In [7]:
NUM_BINGS = 100

In [8]:
cond_counts = c.data.groupby('name').size().sort_values(ascending=False)
to_bing = list(cond_counts[:NUM_BINGS].index)

In [ ]:
bing_res = {}
for cur_term in to_bing:
    feat_doc, feat_links, feat_wiki = bing.bing(cur_term)
    bing_res[cur_term] = {
        'doc': feat_doc,
        'links': feat_links,
        'wiki': feat_wiki
    }
    bing.do_wait()  # don't forget to wait !!

### next 900

In [17]:
to_bing = list(cond_counts[100:1000].index)

In [19]:
for cur_term in tqdm(to_bing):
    feat_doc, feat_links, feat_wiki = bing.bing(cur_term, do_print=False)
    bing_res[cur_term] = {
        'doc': feat_doc,
        'links': feat_links,
        'wiki': feat_wiki
    }
    bing.do_wait(multiple=0.5)  # don't forget to wait !!

100%|██████████| 900/900 [32:13<00:00,  2.15s/it]


In [21]:
pickle.dump(bing_res, open('raw_data/top1000.p', 'wb'))

### Top 10000

In [22]:
to_bing = list(cond_counts[1000:10000].index)

In [23]:
for cur_term in tqdm(to_bing):
    feat_doc, feat_links, feat_wiki = bing.bing(cur_term, do_print=False)
    bing_res[cur_term] = {
        'doc': feat_doc,
        'links': feat_links,
        'wiki': feat_wiki
    }
    bing.do_wait(multiple=0.5)  # don't forget to wait !!

100%|██████████| 9000/9000 [5:31:04<00:00,  2.21s/it]  


In [24]:
pickle.dump(bing_res, open('raw_data/top10000.p', 'wb'))

### Everything else:

In [26]:
to_bing = list(cond_counts[10000:].index)

In [ ]:
for cur_term in tqdm(to_bing):
    feat_doc, feat_links, feat_wiki = bing.bing(cur_term, do_print=False)
    bing_res[cur_term] = {
        'doc': feat_doc,
        'links': feat_links,
        'wiki': feat_wiki
    }
    bing.do_wait(multiple=0.5)  # don't forget to wait !!

In [29]:
pickle.dump(bing_res, open('raw_data/all_bing.p', 'wb'))

In [ ]:
## LOAD !!!
#bing_res = pickle.load(open('raw_data/all_bing.p', 'rb'))

In [48]:
len(bing_res)

44713

In [49]:
list(bing_res.keys())[-1]

'Primary Molar Pulpotomy'

In [50]:
to_bing = list(cond_counts[44713:].index)

In [ ]:
for cur_term in tqdm(to_bing):
    feat_doc, feat_links, feat_wiki = bing.bing(cur_term, do_print=False)
    bing_res[cur_term] = {
        'doc': feat_doc,
        'links': feat_links,
        'wiki': feat_wiki
    }
    bing.do_wait(multiple=0.5)  # don't forget to wait !!




  0%|          | 0/20431 [00:00<?, ?it/s]


  0%|          | 1/20431 [00:02<14:57:34,  2.64s/it]


  0%|          | 2/20431 [00:04<14:04:09,  2.48s/it]


  0%|          | 3/20431 [00:06<13:09:20,  2.32s/it]


  0%|          | 4/20431 [00:08<12:21:21,  2.18s/it]


  0%|          | 5/20431 [00:10<12:15:27,  2.16s/it]


  0%|          | 6/20431 [00:12<12:31:29,  2.21s/it]


  0%|          | 7/20431 [00:16<15:12:24,  2.68s/it]


  0%|          | 8/20431 [00:18<14:00:34,  2.47s/it]


  0%|          | 9/20431 [00:20<13:35:46,  2.40s/it]


  0%|          | 10/20431 [00:23<13:03:05,  2.30s/it]


  0%|          | 11/20431 [00:25<12:33:37,  2.21s/it]


  0%|          | 12/20431 [00:27<12:50:26,  2.26s/it]


  0%|          | 13/20431 [00:29<12:44:41,  2.25s/it]


  0%|          | 14/20431 [00:31<12:37:57,  2.23s/it]


  0%|          | 15/20431 [00:33<12:25:44,  2.19s/it]


  0%|          | 16/20431 [00:36<12:27:02,  2.20s/it]


  0%|          | 17/20431 [00:38<12:20:26,  2.18s/it]


  0%|     

  1%|          | 125/20431 [04:52<13:08:39,  2.33s/it]


  1%|          | 126/20431 [04:54<12:47:33,  2.27s/it]


  1%|          | 127/20431 [04:56<12:30:18,  2.22s/it]


  1%|          | 128/20431 [04:58<12:31:27,  2.22s/it]


  1%|          | 129/20431 [05:01<12:30:53,  2.22s/it]


  1%|          | 130/20431 [05:03<13:05:14,  2.32s/it]


  1%|          | 131/20431 [05:06<13:17:23,  2.36s/it]


  1%|          | 132/20431 [05:08<13:35:41,  2.41s/it]


  1%|          | 133/20431 [05:11<13:52:40,  2.46s/it]


  1%|          | 134/20431 [05:14<14:50:31,  2.63s/it]


  1%|          | 135/20431 [05:16<14:44:51,  2.62s/it]


  1%|          | 136/20431 [05:19<14:10:16,  2.51s/it]


  1%|          | 137/20431 [05:23<18:08:28,  3.22s/it]


  1%|          | 138/20431 [05:26<16:58:09,  3.01s/it]


  1%|          | 139/20431 [05:30<18:18:29,  3.25s/it]


  1%|          | 140/20431 [05:32<16:57:42,  3.01s/it]


  1%|          | 141/20431 [05:35<16:07:48,  2.86s/it]


  1%|          | 142/20431 [05:

  1%|          | 249/20431 [09:56<14:00:36,  2.50s/it]


  1%|          | 250/20431 [09:58<12:54:08,  2.30s/it]


  1%|          | 251/20431 [10:00<11:49:23,  2.11s/it]


  1%|          | 252/20431 [10:02<13:06:41,  2.34s/it]


  1%|          | 253/20431 [10:04<12:02:16,  2.15s/it]


  1%|          | 254/20431 [10:07<12:28:47,  2.23s/it]


  1%|          | 255/20431 [10:09<12:18:12,  2.20s/it]


  1%|▏         | 256/20431 [10:12<13:59:08,  2.50s/it]


  1%|▏         | 257/20431 [10:14<13:13:41,  2.36s/it]


  1%|▏         | 258/20431 [10:15<11:49:51,  2.11s/it]


  1%|▏         | 259/20431 [10:18<11:46:17,  2.10s/it]


  1%|▏         | 260/20431 [10:20<12:16:05,  2.19s/it]


  1%|▏         | 261/20431 [10:23<13:11:52,  2.36s/it]


  1%|▏         | 262/20431 [10:25<13:55:17,  2.48s/it]


  1%|▏         | 263/20431 [10:31<18:27:22,  3.29s/it]


  1%|▏         | 264/20431 [10:32<16:01:19,  2.86s/it]


  1%|▏         | 265/20431 [10:35<15:25:05,  2.75s/it]


  1%|▏         | 266/20431 [10:

  2%|▏         | 373/20431 [14:40<12:18:12,  2.21s/it]


  2%|▏         | 374/20431 [14:42<12:49:29,  2.30s/it]


  2%|▏         | 375/20431 [14:44<12:23:03,  2.22s/it]


  2%|▏         | 376/20431 [14:46<11:38:56,  2.09s/it]


  2%|▏         | 377/20431 [14:48<11:05:17,  1.99s/it]


  2%|▏         | 378/20431 [14:50<11:37:19,  2.09s/it]


  2%|▏         | 379/20431 [14:53<12:22:35,  2.22s/it]


  2%|▏         | 380/20431 [14:55<13:07:32,  2.36s/it]


  2%|▏         | 381/20431 [14:58<13:36:17,  2.44s/it]


  2%|▏         | 382/20431 [15:01<14:44:49,  2.65s/it]


  2%|▏         | 383/20431 [15:04<15:14:41,  2.74s/it]


  2%|▏         | 384/20431 [15:08<16:30:59,  2.97s/it]


  2%|▏         | 385/20431 [15:10<15:24:33,  2.77s/it]


  2%|▏         | 386/20431 [15:13<15:13:53,  2.74s/it]


  2%|▏         | 387/20431 [15:15<14:17:17,  2.57s/it]


  2%|▏         | 388/20431 [15:17<13:07:56,  2.36s/it]


  2%|▏         | 389/20431 [15:19<12:36:09,  2.26s/it]


  2%|▏         | 390/20431 [15:

  2%|▏         | 497/20431 [19:18<12:06:29,  2.19s/it]


  2%|▏         | 498/20431 [19:20<12:49:26,  2.32s/it]


  2%|▏         | 499/20431 [19:22<11:26:10,  2.07s/it]


  2%|▏         | 500/20431 [19:24<11:22:07,  2.05s/it]


  2%|▏         | 501/20431 [19:26<11:31:30,  2.08s/it]


  2%|▏         | 502/20431 [19:28<11:28:59,  2.07s/it]


  2%|▏         | 503/20431 [19:30<10:42:40,  1.93s/it]


  2%|▏         | 504/20431 [19:32<11:02:23,  1.99s/it]


  2%|▏         | 505/20431 [19:35<12:21:43,  2.23s/it]


  2%|▏         | 506/20431 [19:36<11:31:16,  2.08s/it]


  2%|▏         | 507/20431 [19:39<12:49:20,  2.32s/it]


  2%|▏         | 508/20431 [19:41<12:25:08,  2.24s/it]


  2%|▏         | 509/20431 [19:44<12:43:15,  2.30s/it]


  2%|▏         | 510/20431 [19:46<12:25:00,  2.24s/it]


  3%|▎         | 511/20431 [19:48<12:34:04,  2.27s/it]


  3%|▎         | 512/20431 [19:50<12:26:03,  2.25s/it]


  3%|▎         | 513/20431 [19:53<12:21:30,  2.23s/it]


  3%|▎         | 514/20431 [19:

...

  5%|▌         | 1115/20431 [42:02<12:48:11,  2.39s/it]


  5%|▌         | 1116/20431 [42:04<11:52:10,  2.21s/it]


  5%|▌         | 1117/20431 [42:06<11:50:46,  2.21s/it]


  5%|▌         | 1118/20431 [42:09<12:24:02,  2.31s/it]


  5%|▌         | 1119/20431 [42:12<13:55:34,  2.60s/it]


  5%|▌         | 1120/20431 [42:14<12:51:27,  2.40s/it]


  5%|▌         | 1121/20431 [42:17<12:59:37,  2.42s/it]


  5%|▌         | 1122/20431 [42:18<11:59:43,  2.24s/it]


  5%|▌         | 1123/20431 [42:20<11:29:57,  2.14s/it]


  6%|▌         | 1124/20431 [42:23<11:34:33,  2.16s/it]


  6%|▌         | 1125/20431 [42:24<11:08:00,  2.08s/it]


  6%|▌         | 1126/20431 [42:27<11:35:45,  2.16s/it]


  6%|▌         | 1127/20431 [42:29<12:01:00,  2.24s/it]


  6%|▌         | 1128/20431 [42:32<12:09:16,  2.27s/it]


  6%|▌         | 1129/20431 [42:34<12:16:26,  2.29s/it]


  6%|▌         | 1130/20431 [42:36<11:54:31,  2.22s/it]


  6%|▌         | 1131/20431 [42:38<11:47:53,  2.20s/it]


  6%|▌        

  6%|▌         | 1237/20431 [46:41<13:21:39,  2.51s/it]


  6%|▌         | 1238/20431 [46:43<12:17:50,  2.31s/it]


  6%|▌         | 1239/20431 [46:45<12:06:00,  2.27s/it]


  6%|▌         | 1240/20431 [46:48<12:38:06,  2.37s/it]


  6%|▌         | 1241/20431 [46:50<12:11:14,  2.29s/it]


  6%|▌         | 1242/20431 [46:52<12:02:53,  2.26s/it]


  6%|▌         | 1243/20431 [46:54<11:39:52,  2.19s/it]


  6%|▌         | 1244/20431 [46:56<11:31:32,  2.16s/it]


  6%|▌         | 1245/20431 [46:58<10:49:42,  2.03s/it]


  6%|▌         | 1246/20431 [47:00<10:45:58,  2.02s/it]


  6%|▌         | 1247/20431 [47:02<11:04:26,  2.08s/it]


  6%|▌         | 1248/20431 [47:04<10:24:24,  1.95s/it]


  6%|▌         | 1249/20431 [47:06<11:03:10,  2.07s/it]


  6%|▌         | 1250/20431 [47:08<10:28:57,  1.97s/it]


  6%|▌         | 1251/20431 [47:11<11:48:21,  2.22s/it]


  6%|▌         | 1252/20431 [47:13<12:23:24,  2.33s/it]


  6%|▌         | 1253/20431 [47:16<12:53:19,  2.42s/it]


  6%|▌        

  7%|▋         | 1359/20431 [51:18<10:36:48,  2.00s/it]


  7%|▋         | 1360/20431 [51:20<10:57:17,  2.07s/it]


  7%|▋         | 1361/20431 [51:23<12:22:15,  2.34s/it]


  7%|▋         | 1362/20431 [51:25<11:47:47,  2.23s/it]


  7%|▋         | 1363/20431 [51:27<11:24:11,  2.15s/it]


  7%|▋         | 1364/20431 [51:29<10:45:47,  2.03s/it]


  7%|▋         | 1365/20431 [51:31<10:35:12,  2.00s/it]


  7%|▋         | 1366/20431 [51:33<10:54:21,  2.06s/it]


  7%|▋         | 1367/20431 [51:35<10:48:42,  2.04s/it]


  7%|▋         | 1368/20431 [51:37<10:18:55,  1.95s/it]


  7%|▋         | 1369/20431 [51:39<10:42:46,  2.02s/it]


  7%|▋         | 1370/20431 [51:41<11:10:51,  2.11s/it]


  7%|▋         | 1371/20431 [51:43<11:30:35,  2.17s/it]


  7%|▋         | 1372/20431 [51:46<11:43:36,  2.22s/it]


  7%|▋         | 1373/20431 [51:48<12:12:20,  2.31s/it]


  7%|▋         | 1374/20431 [51:52<13:43:31,  2.59s/it]


  7%|▋         | 1375/20431 [51:54<13:07:32,  2.48s/it]


  7%|▋        

  7%|▋         | 1481/20431 [55:50<11:59:42,  2.28s/it]


  7%|▋         | 1482/20431 [55:52<11:54:45,  2.26s/it]


  7%|▋         | 1483/20431 [55:54<11:38:56,  2.21s/it]


  7%|▋         | 1484/20431 [55:56<11:56:12,  2.27s/it]


  7%|▋         | 1485/20431 [55:59<12:09:11,  2.31s/it]


  7%|▋         | 1486/20431 [56:01<11:37:04,  2.21s/it]


  7%|▋         | 1487/20431 [56:03<11:23:23,  2.16s/it]


  7%|▋         | 1488/20431 [56:04<10:46:48,  2.05s/it]


  7%|▋         | 1489/20431 [56:06<10:37:06,  2.02s/it]


  7%|▋         | 1490/20431 [56:08<10:13:32,  1.94s/it]


  7%|▋         | 1491/20431 [56:10<10:14:35,  1.95s/it]


  7%|▋         | 1492/20431 [56:13<10:55:23,  2.08s/it]


  7%|▋         | 1493/20431 [56:15<10:54:57,  2.08s/it]


  7%|▋         | 1494/20431 [56:16<10:29:46,  2.00s/it]


  7%|▋         | 1495/20431 [56:19<10:45:45,  2.05s/it]


  7%|▋         | 1496/20431 [56:21<11:10:36,  2.12s/it]


  7%|▋         | 1497/20431 [56:24<11:58:17,  2.28s/it]


  7%|▋        

  8%|▊         | 1603/20431 [1:00:15<11:42:55,  2.24s/it]


  8%|▊         | 1604/20431 [1:00:17<11:47:25,  2.25s/it]


  8%|▊         | 1605/20431 [1:00:19<11:37:03,  2.22s/it]


  8%|▊         | 1606/20431 [1:00:21<11:19:16,  2.17s/it]


  8%|▊         | 1607/20431 [1:00:24<13:18:37,  2.55s/it]


  8%|▊         | 1608/20431 [1:00:26<12:25:52,  2.38s/it]


  8%|▊         | 1609/20431 [1:00:29<12:27:02,  2.38s/it]


  8%|▊         | 1610/20431 [1:00:31<12:11:13,  2.33s/it]


  8%|▊         | 1611/20431 [1:00:33<11:54:44,  2.28s/it]


  8%|▊         | 1612/20431 [1:00:35<11:06:27,  2.12s/it]


  8%|▊         | 1613/20431 [1:00:37<11:06:39,  2.13s/it]


  8%|▊         | 1614/20431 [1:00:39<11:29:42,  2.20s/it]


  8%|▊         | 1615/20431 [1:00:42<11:54:49,  2.28s/it]


  8%|▊         | 1616/20431 [1:00:44<11:15:06,  2.15s/it]


  8%|▊         | 1617/20431 [1:00:46<11:39:40,  2.23s/it]


  8%|▊         | 1618/20431 [1:00:48<11:04:18,  2.12s/it]


  8%|▊         | 1619/20431 [1:00:50<10: